# Rescale rat model to match Long-Evans rat skeletal parameters

### Changes include
* Global scaling to approximate average rat size.
* Rescaling of long bones to measured lengths to preserve proportions. 

In [1]:
from dm_control import mjcf
from dm_control import viewer
from dm_control import mujoco
from dm_control import suite
import numpy as np
import re

## Define useful parameters and functions

In [2]:
base_model_path = '/home/diego/code/olveczky/dm/stac/models/to_tune/rat_june24.xml'
# temp_model_path is the model used by .../dm_control/suite/rat.py for easy iteration.
temp_model_path = '/home/diego/.envs/mujoco200_3.7/lib/python3.6/site-packages/dm_control/suite/rat_temp.xml'

def view_model():
    # Load an environment from the Control Suite.
    env = suite.load(domain_name="rat", task_name="stand")
    # Launch the viewer application.
    viewer.launch(env)

def load_model(model_path):
    return mjcf.from_path(model_path)

def write_model(model, save_path):
    s = model.to_xml_string()
    s = re.sub('rat_skin.*skn', 'rat_skin.skn', s)
    with open(save_path , 'w') as f:
        f.write(s)

# Save measured data (mm) and the bone-site pairs they correspond to 
bone_lengths = {'humerus': 30.0,
                'radius': 29.6,
                'femur': 36.5,
                'tibia': 42.8,
                'metatarsal': 23.4,
                'hand': 9.0}
joint_pairs = {'humerus': ['shoulder_L', 'elbow_L'],
               'radius': ['elbow_L', 'wrist_L'],
               'femur': ['hip_L', 'knee_L'],
               'tibia': ['knee_L', 'ankle_L'],
               'metatarsal': ['ankle_L', 'toe_L'],
               'hand': ['wrist_L', 'finger_L']}

skull_width = 38.8
skull_length = 57

skull_dims = {'length': [47.4], 'width': [20.2]}
skull_pairs = {'length': ['head', 'skull_T0_collision'],
              'width': ['eye_R_collision', 'eye_L_collision']}



base_model = load_model(base_model_path)
print(base_model.to_xml_string())
write_model(base_model, temp_model_path)
view_model()

<mujoco model="rat">
  <compiler angle="radian"/>
  <size njmax="2000" nconmax="1000"/>
  <visual>
    <quality shadowsize="2048"/>
  </visual>
  <statistic meansize="0.003"/>
  <default>
    <default class="/">
      <mesh scale="1 1 1"/>
      <joint limited="true" solreflimit="0.01 1" solimplimit="0.98999999999999999 0.99990000000000001 0" armature="6e-08" damping="0.0001"/>
      <geom friction="0.69999999999999996 0.0050000000000000001 0.0001" solref="0.0050000000000000001 1"/>
      <site group="3" rgba="0 0 1 1"/>
      <general ctrllimited="true" ctrlrange="-1 1" gainprm="0.01"/>
      <default class="bone">
        <geom contype="0" conaffinity="0" group="0" rgba="0.5 0.5 0.5 1" density="1100.0"/>
      </default>
      <default class="collision">
        <geom contype="1" conaffinity="1" condim="1" group="2"/>
        <default class="collision_primitive">
          <geom type="capsule" group="2" rgba="0 0.5 0.5 1" density="500.0"/>
          <default class="collision_primitiv

## View the base model

In [3]:
# print([p for p in env.physics.named.model._asdict().keys() if 'jnt' in p])
# env.physics.named.data.xpos

In [4]:
def get_bone_distance(physics, joint_pair):
    joint0 = physics.named.data.site_xpos[joint_pair[0]].copy()
    joint1 = physics.named.data.site_xpos[joint_pair[1]].copy()
    length = np.sqrt(np.sum((joint0 - joint1)**2))
    return length

def get_bone_ratios(bone_dict):
    n_bones = len(bone_dict.keys())
    ratio_mat = np.zeros((n_bones,n_bones))
    ratio_dict = {}
    for i, (bone0, length0) in enumerate(bone_dict.items()):
        for j, (bone1, length1) in enumerate(bone_dict.items()):
            ratio = length0/length1
            ratio_mat[i, j] = ratio
            ratio_dict[bone0 + '-' + bone1] = ratio
            
    return ratio_dict, ratio_mat

def get_skull_dims(physics):
    atlas_pos = physics.named.data.xpos['vertebra_atlant'].copy()
    print(atlas_pos)
    T0_pos = physics.named.data.geom_xpos['skull_T0_collision'].copy()
    length = np.sqrt(np.sum((atlas_pos - T0_pos)**2))*1000
    
    eyeL_pos = physics.named.data.geom_xpos['eye_L_collision'].copy()
    eyeR_pos = physics.named.data.geom_xpos['eye_R_collision'].copy()
    width = np.sqrt(np.sum((eyeL_pos - eyeR_pos)**2))*1000
    return {'length': length, 'width': width}


## Apply global downscaling, and long-bone length matching

In [5]:
model = load_model(base_model_path)
write_model(base_model, temp_model_path)
# Globally scale down the model
def scale_model(model, global_scale_ratio=.82):
    for g in model.find_all('geom'):
        if g.pos is not None and 'eye' not in g.name:
            g.pos *= global_scale_ratio
    for b in model.find_all('body'):
        if b.pos is not None and 'eye' not in g.name:
            b.pos *= global_scale_ratio
    for s in model.find_all('site'):
        if s.pos is not None and 'eye' not in g.name:
            s.pos *= global_scale_ratio
    return model
model = scale_model(model)
write_model(model, temp_model_path)

env = suite.load(domain_name="rat", task_name="stand")
model_lengths = {k: get_bone_distance(env.physics, jp)*1000 for k, jp in joint_pairs.items()}
length_difference = {k: bone_lengths[k] - model_lengths[k] for k in model_lengths.keys()}
bone_ratios, bone_ratio_mat = get_bone_ratios(bone_lengths)
model_ratios, model_ratio_mat = get_bone_ratios(model_lengths)
print('Model lengths:', model_lengths)
print('Bone lengths:', bone_lengths)
ratio = [bone_lengths[k]/model_lengths[k] for k in model_lengths.keys()]
print('Ratio of bone_lengths to model_lengths', ratio)

# Scale particular arm and leg joints to match literature
def scale_arms_and_legs(model):
    env = suite.load(domain_name="rat", task_name="stand")
    model_lengths = {k: get_bone_distance(env.physics, jp)*1000 for k, jp in joint_pairs.items()}
    ratio = [bone_lengths[k]/model_lengths[k] for k in model_lengths.keys()]
    model_name_pairs = {'humerus': ['elbow', 'humerus', 'lower_arm'],
                   'radius': ['wrist', 'radius', 'ulna', 'hand'],
                   'femur': ['knee', 'upper_leg_L0_collision', 'upper_leg_R0_collision', 'lower_leg'],
                   'tibia': ['ankle', 'foot'],
                   'metatarsal': ['toe'],
                   'hand': ['finger', 'hand_L_collision', 'hand_R_collision']}
    for i, (bone, model_id) in enumerate(model_name_pairs.items()):
        for g in model.find_all('geom'):
            if any(part in g.name for part in model_id):
                if bone == 'radius' and any(part in g.name for part in ['hand_L_collision', 'hand_R_collision']):
                    continue
                if g.pos is not None:
                    g.pos *= ratio[i]
                if g.size is not None:
                    g.size *= ratio[i]
        for b in model.find_all('body'):
            if any(part in b.name for part in model_id):
                if b.pos is not None:
                    b.pos *= ratio[i]
        for s in model.find_all('site'):
            if any(part in s.name for part in model_id):
                if s.pos is not None:
                    s.pos *= ratio[i]
    return model
model = scale_arms_and_legs(model)
write_model(model, temp_model_path)


# # Scale particular arm and leg joints to match literature
# def scale_skull(model):
#     env = suite.load(domain_name="rat", task_name="stand")
#     model_dims = get_skull_dims(env.physics)
#     ratio = [skull_dims[k]/model_dims[k] for k in model_dims.keys()]
#     print('model_dims', model_dims)
#     print(ratio)
#     model_name_pairs = {'length': ['jaw', 'skull', 'eye'],
#                        'width': ['jaw', 'skull', 'eye']}
#     for i, (bone, model_id) in enumerate(model_name_pairs.items()):
#         for g in model.find_all('geom'):
#             if any(part in g.name for part in model_id):
#                 if g.pos is not None:
#                     g.pos[i] *= ratio[i]
#                 if 'eye' in g.name:
#                     continue
#                 if g.size is not None:
#                     g.size[i] *= ratio[i]
#         for b in model.find_all('body'):
#             if any(part in b.name for part in model_id):
#                 if b.pos is not None:
#                     b.pos[i] *= ratio[i]
#         for s in model.find_all('site'):
#             if any(part in s.name for part in model_id):
#                 if s.pos is not None:
#                     s.pos[i] *= ratio[i]
#     return model
# model = scale_skull(model)
# write_model(model, temp_model_path)

Model lengths: {'humerus': 24.599999999999998, 'radius': 24.271999999999995, 'femur': 29.930000000000003, 'tibia': 35.096000000000004, 'metatarsal': 19.187999999999995, 'hand': 7.379999999999997}
Bone lengths: {'humerus': 30.0, 'radius': 29.6, 'femur': 36.5, 'tibia': 42.8, 'metatarsal': 23.4, 'hand': 9.0}
Ratio of bone_lengths to model_lengths [1.2195121951219514, 1.2195121951219516, 1.2195121951219512, 1.219512195121951, 1.2195121951219514, 1.2195121951219516]


## Check to make sure everything worked as planned

In [6]:
env = suite.load(domain_name="rat", task_name="stand")
model_lengths = {k: get_bone_distance(env.physics, jp)*1000 for k, jp in joint_pairs.items()}
length_difference = {k: bone_lengths[k] - model_lengths[k] for k in model_lengths.keys()}
bone_ratios, bone_ratio_mat = get_bone_ratios(bone_lengths)
model_ratios, model_ratio_mat = get_bone_ratios(model_lengths)
print('Model lengths:', model_lengths)
print('Bone lengths:', bone_lengths)
ratio = [bone_lengths[k]/model_lengths[k] for k in model_lengths.keys()]
print('Ratio of bone_lengths to model_lengths', ratio)


# model_dims = get_skull_dims(env.physics)
# ratio = [skull_dims[k]/model_dims[k] for k in model_dims.keys()]
# print('Ratio of bone_lengths to model_lengths', ratio)

Model lengths: {'humerus': 30.0, 'radius': 29.600000000000016, 'femur': 36.5, 'tibia': 42.79999999999999, 'metatarsal': 23.4, 'hand': 8.999999999999998}
Bone lengths: {'humerus': 30.0, 'radius': 29.6, 'femur': 36.5, 'tibia': 42.8, 'metatarsal': 23.4, 'hand': 9.0}
Ratio of bone_lengths to model_lengths [1.0, 0.9999999999999996, 1.0, 1.0000000000000002, 1.0, 1.0000000000000002]


## View the model

In [7]:
view_model()